# Narupa Servers

In this notebook, we explore what goes into a Narupa server, and how to choose a flavour for a given application.

This gets into some of the nuts and bolts of how Narupa works under the hood, including some GRPC details. While understanding GRPC is not necessary, it [will help](http://grpc.io).

## The Narupa Server Architecture

Narupa uses a client-server model. This means we have some stuff running on a server, and some stuff running on a client, and they talk to each other. It's exactly how most apps you run on your phone work, and websites:

![Narupa Client Server](./images/narupa_client_server.png)

In Narupa, the server is in charge of setting up, managing and running simulations and models. This could be trajectory serving, interactive molecular dynamics, or minimizing structures produced in the builder. 

The client is any application that connects to the server. We generally have two types of client, a VR client for viewing and manipulating simulations in VR, and python clients for experimentation and testing. However, our framework is flexible enough that we could have other types of clients, such as web apps.

### GRPC

The clients and the server talk to each other using [GRPC](grpc.io). GRPC is an application level communication protocol based on the idea of Remote Procedure Call. A client calls *procedures* (methods) on servers *remotely*, and then the server returns the result at some point in the future. GRPC handles all of the details of sending packets over the network for us, letting us focus on building the application. 

GRPC has a number of features that are good for us: 

* High performance. 
* Ability to do streaming (send lots of data from the server) and bidrectional streaming (send lots of data in both directions between client and server). 
* Works in many languages, including python and C#.

### GRPC Services

In GRPC, we create services that lay out a definition of what a server can do. Combining these services produces an application. For example, the Narupa IMD server uses a TrajectoryService, an IMDService, a MultiplayerService, a CommandService and a StateService: 

![Narupa IMD Application](./images/narupa_imd_server.png)

In the above image, we left out the Command service and State service as implicit. 
 
Let's look at the Command Service (see the [commands notebook](./commands_and_state.ipynb)), which is used in Narupa to run arbitrary commands. The definition is written in a protobuf file.

```proto
package narupa.protocol.command;

service Command {

    /* Get a list of all the commands available on this service */
    rpc GetCommands (GetCommandsRequest) returns (GetCommandsReply) {}

    /* Runs a command on the service */
    rpc RunCommand (CommandMessage) returns (CommandReply) {}
}

message GetCommandsRequest {

}

message GetCommandsReply{
    repeated CommandMessage commands = 1;
}

message CommandReply {
    google.protobuf.Struct result = 1;
}

message CommandMessage {
    string name = 1;
    google.protobuf.Struct arguments = 2;
}
```


So we've defined a service called `Command`, which has two methods: 

* `RunCommand` - takes a `CommandMessage`, consisting of a `name` and a dictionary-like Struct of `arguments`, and returns a `CommandReply`, which is contains a dictionary of results. 
* `GetCommands` - takes a `GetCommandsRequest`, which is an empty message, and returns as `GetCommandsReply` which is a list (`repeated`) of `CommandMessages`. The `CommandMessage`, in turn, consists of a `name`, which tells us the name of the command, and a Struct (a JSON-style dictionary) of default `arguments`. 


To build a server, we take these service definitions and write the code that actually does what this specification says it should do. Then for clients, we just *call* these functions, knowing that they will produce the results we need. 

## The Server Stack

Narupa servers are just a collection of these GRPC services, with some bells and whistles attached.

The Narupa libraries have a lot of servers lying around:

In [ ]:
from narupa.app import NarupaApplicationServer, NarupaFrameApplication, NarupaImdApplication
from narupa.core import NarupaServer, GrpcServer
from narupa.trajectory import FrameServer
from narupa.imd import ImdServer

What do these all do? What should we be using?

In what follows, we'll work our way up to high-level application servers, starting from the bottom of the stack with the simplest server

# TLDR

If you want to send Narupa Frames, e.g. trajectory viewing, you probably want the [NarupaFrameApplication](https://narupa.readthedocs.io/en/latest/python/narupa.app.frame_app.html).

If you want to set up an interactive molecular dynamics simulation, you probably want the [NarupaImdApplication](https://narupa.readthedocs.io/en/latest/python/narupa.app.imd_app.html).

### GRPC Server

It all starts with the Grpc Server, which forms the base of all other servers.

In [ ]:
grpc_server = GrpcServer(address='localhost', port=0)

By itself, this server doesn't do much other than set up the underlying [gRPC server](http://grpc.io) with a few little helpers:

In [ ]:
print(grpc_server.__doc__)

In [ ]:
[x for x in dir(grpc_server) if not x.startswith('_')]

Mainly, it provides a method to gracefully close, access to the address and port the server is running on, and the ability to set up new gRPC services. 

If you just want to run a python GRPC server with a couple of little helpers, this is the one for you.

In practice, Narupa servers always want commands and state synchronisation, so let's add those. 

In [ ]:
from narupa.command import CommandService
from narupa.state.state_service import StateService

command_service = CommandService()
state_service = StateService()

These are the python implementations of the Command and State services. If we add them to the server, it will gain the ability to run commands and synchronise state (see the [commands and state](./commands_and_state.ipynb) notebook).

In [ ]:
grpc_server.add_service(command_service)
grpc_server.add_service(state_service)

In [ ]:
def hello():
    print('hi!')

command_service.register_command('hello', hello)

Let's check that worked

In [ ]:
from narupa.core import NarupaClient

with NarupaClient.establish_channel(address=grpc_server.address, port=grpc_server.port) as client:
    client.run_command('hello')

Cool! We've just created a functioning Narupa server! If you wanted to write your own GRPC services, you could add them with the same methodology, adding them to the server with `add_service`.

In [ ]:
grpc_server.close()

### Narupa Server

Since we almost always want commands and state synchronisation, we've created the Narupa Server object that does exactly that, so you don't have to type the above. 

In [ ]:
narupa_server = NarupaServer(address='localhost', port=0)

In [ ]:
print(narupa_server.__doc__)

In [ ]:
narupa_server.register_command('narupa_hello', hello)

In [ ]:
with NarupaClient.establish_channel(address=narupa_server.address, port=narupa_server.port) as client:
    client.run_command('narupa_hello')

That's the Narupa server. It's just a GRPC server with the command and state service added. 

### Frame Server

Most Narupa applications want to transmit some sort of simulation data, i.e. Frames, to clients. For that, we need the frame publishing service. Let's add that to our server:

In [ ]:
from narupa.trajectory import FramePublisher
from narupa.trajectory import FrameData

In [ ]:
frame_publisher = FramePublisher()

In [ ]:
narupa_server.add_service(frame_publisher)

In [ ]:
frame = FrameData()
frame.values['hello'] = 'hello'
frame_publisher.send_frame(0, frame)

Let's check that we can connect and receive frames. The `NarupaImdClient` class is a python client that knows how to receive frames.

In [ ]:
from narupa.app import NarupaImdClient
import time 

with NarupaImdClient.connect_to_single_server(address=narupa_server.address, port=narupa_server.port) as client:
    client.subscribe_to_frames()
    client.wait_until_first_frame()
    print(client.first_frame)

In [ ]:
narupa_server.close()

This is now a functioning frame server! If we wanted, we could connect to this from VR (if we sent something that looked like a molecule). See the [frames](./frames.ipynb) example notebook for more details on setting up Narupa frames.

Since this is common functionality, we wrap this in the `FrameServer`. Similarly, we do the same for multiplayer and IMD with the `MultiplayerServer` and `ImdServer`:

In [ ]:
from narupa.trajectory import FrameServer

In [ ]:
frame_server = FrameServer(address='localhost', port=0)

In [ ]:
frame_server.send_frame(0, frame)

In [ ]:
with NarupaImdClient.connect_to_single_server(address=frame_server.address, port=frame_server.port) as client:
    client.subscribe_to_frames()
    client.wait_until_first_frame()
    print(client.first_frame)

### Multiplayer

The server itself is multiplayer agnostic--it provides the ability for clients to coordinate data via the State Service, but it doesn't understand how they are doing it. Clients subscribe to updates from the State Service, and send their own value updates:

In [ ]:
with NarupaImdClient.connect_to_single_server(address=frame_server.address, port=frame_server.port) as client:
    client.set_shared_value('a', 2)
    with NarupaImdClient.connect_to_single_server(address=frame_server.address, port=frame_server.port) as second_client:
        second_client.subscribe_multiplayer()
        time.sleep(0.05) # Wait for messages to be received.
        print(second_client.latest_multiplayer_values)

### Discovery (advertising and finding services on a network)

This is good if we know the address and port to connect to, but can we make it so we can autoconnect, or find it on the network?

Yes! We can manually set up a Discovery server, so our server can be found on the local network.

In [ ]:
from narupa.essd import DiscoveryServer, ServiceHub, DiscoveryClient

In [ ]:
discovery_server = DiscoveryServer()

We define the service hub, specifying what services are available and which port they are running at.

In [ ]:
service_hub = ServiceHub(name="My Frame Server", address=frame_server.address)
service_hub.add_service("trajectory", frame_server.port)
service_hub.add_service("multiplayer",frame_server.port)

In [ ]:
discovery_server.register_service(service_hub)

The discovery server will now be broadcasting the existence of the server! Let's search for it:

In [ ]:
discovery_client = DiscoveryClient()

In [ ]:
import pprint # pretty print
pprint.pprint(list(discovery_client.search_for_services(search_time=1.0)))

You may find some other servers that are running on the network, but hopefully your server was found!

Now we can use the client's autoconnect functionality (note this may produce unexpected results if you've got multiple servers on the network):

In [ ]:
with NarupaImdClient.autoconnect() as client:
    client.subscribe_to_frames()
    print(client.first_frame)

In [ ]:
frame_server.close()

## The Application Servers

Phew, that was quite a lot of work! Luckily, we have a handy wrapper that does all of that for you, the `NarupaApplicationServer`.

In [ ]:
print(NarupaApplicationServer.__doc__)

The `NarupaFrameApplication` and `NarupaImdApplication` classes inherit from the `NarupaApplicationServer`. The former adds frame support, while the latter adds both frame support and IMD support. Let's try it out.

In [ ]:
imd_app = NarupaImdApplication.basic_server(name="My First Narupa Imd App", port=0)

In [ ]:
[x for x in dir(imd_app) if not x.startswith('_')]

If you were writing your own interactive molecular dynamics application, this is all you need. You can send frames, and you'll receive interactions that you can apply to your MD:

Below, we simulate a client connecting, receiving a frame and sending an (empty) interaction.

In [ ]:
imd_app.frame_publisher.send_frame(0, frame)

In [ ]:
from narupa.imd.particle_interaction import ParticleInteraction 

with NarupaImdClient.connect_to_single_server(port=imd_app.port) as client:
    client.subscribe_to_frames()
    interaction_id = client.start_interaction()
    client.update_interaction(interaction_id, ParticleInteraction())
    time.sleep(0.05)
    print(f'Active interactions: {imd_app.imd.active_interactions}')
    print(f'Frame Received: {client.first_frame}')

In [ ]:
imd_app.close()

## Summary 

In this notebook we've gone from the basic GRPC server all the way up to a full interactive molecular dynamics server with multiplayer, commands, and discovery. 

The final example is how applications in Narupa are actually built. For example, this is a sketch of how our Narupa ASE server works:

![Narupa ASE](./images/narupa-ase.png)

With these examples, combining frames, multiplayer and commands, you can build all sorts of things.

## Next Steps

* See more examples of [commands and state synchronisation](./commands_and_state.ipynb).
* See an example of building a [trajectory viewing application](../mdanalysis/mdanalysis_trajectory.ipynb).